In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [5]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [7]:
garbageCollectionDate[1]

'CentralGoldfieldsShireCouncil-garbage-collection-zones.json'

In [8]:
readInJson(garbageCollectionDate[1]).head(2)

,name,type,features
0,Central Goldfields Garbage Collection Zones,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,Central Goldfields Garbage Collection Zones,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [18]:
df = readInJson(garbageCollectionDate[2])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df.loc[df.geo_nested!=0].shape[0]

0

In [19]:
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo', 'type', 'id', 'geometry', 'geometry_name', 'geo_nested'], axis = 1, inplace = True)

In [20]:
df.iloc[0,0]

{'rub_sched': 'R/2015-01-07/P2W',
 'rec_sched': 'R/2015-01-14/P2W',
 'grn_sched': 'R/2015-01-07/P2W',
 'name': 'W2',
 'desc': 'Area2',
 'rub_cmt': 'Household waste (Red lid bin) - 120 litres',
 'rec_cmt': 'Recycling (Yellow lid bin) - 240 litres (120 litres for units and townhouses)',
 'grn_cmt': 'Green waste (Green lid bin) - 240 litres (120 litres for units and townhouses)',
 'info_url': 'http://www.geelongaustralia.com.au/residents/waste/default.aspx',
 'missed_ph': '(03)52725272'}

In [21]:
newCols = ['landfill_colDay', 
           'recyclable_colDay',
          'green_colDay',]
newColsKeys = ['rub_sched', 
               'rec_sched', 
              'grn_sched']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    df[key] = df.properties.apply(lambda x: x[val])

for i in ['landfill', 'recyclable', 'green']: 
    df[f'{i}_date'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[1])
    df[f'{i}_date'] = pd.to_datetime(df[f'{i}_date'], format='%Y-%m-%d').dt.day_name()
    df[f'{i}_freq'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[-1])
    df[f'{i}_freq'] = df[f'{i}_freq'].apply(lambda x: 'Every Second ' if x == 'P2W' else 'Every ')
    
    df[f'{i}_colDay'] = df[f'{i}_freq'] + df[f'{i}_date']
    
    df.drop([f'{i}_freq', f'{i}_date'], axis = 1, inplace = True)

In [26]:
goldfield = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})

goldfield.to_csv('../GBD_goldfield.csv', index = False)